# AlBi 3
### Übungsblatt 2
*Markus Apel, Mathias Husted*

### Aufgabe 1: Genstruktur I
**a) Warum unterscheidet man in der Genstrukturvorhersage zwischen Eukaryonten und
Prokaryonten?**

Man unterscheidet zwischen Pro- und Eukaryonten, weil die Gene unterschiedlich unterteilt sind.

In **Prokaryonten** werden Gene konsekutiv meist in Operons organisiert, während sie in **Eukaryonten** eine Intron-Exon-Struktur besitzen. Daher wird zusätzlich z.B. die Erkennung von Splice Sites benötigt.

**b) Was sind Indikatoren für proteinkodierende Abschnitte im Genom die sich Genfindungsprogramme zu nutze machen können?**

- Start- und Stoppcodon
- Splice-Sites bei Eukaryonten
- Ribosomal binding site bei Prokaryonten
- Upstream regulatory motifs (Promoter, TATA, etc.)
- Poly-A-Schwanz
- Lange ORFs
- Codonpräferenzen (Leserahmenkonsistenz)

**c) Mit welchem Hobby vergleicht Anders Krogh das Finden von Signalen in Genen und
weshalb?**

Anders Krogh vergleicht es mit Vogelbeobachtung und zieht die Analogie heran, dass man bei der Erkennung von "Vögel" die Umgebungen mit betrachten muss. Bei der Findung von Genen können das Kontextsignale wie benachbarte Exons/Introns sein.

### Aufgabe 2: Genstruktur II
Gegeben ist die Sequenz:

ATGCGCTAGCGGATGATGAGGCCCTAGATGGCC

**a) Finden Sie die Start- und Stopcodons in der DNA Sequenz. Welche ORFs werden
dadurch definiert? Was ist ihre Länge?**

In [ ]:
def find_reading_frames(input):
    # Find start codon ATG
    starting_points = []
    for i, letter in enumerate(input):
        if input[i:i+3] == "ATG":
            starting_points.append(i)
    print(f"Starting points found at {starting_points}")
    
    # Find stop of each point
    stop_codons = ["TAA", "TAG", "TGA"]
    reading_frames = []
    for i in starting_points:
        starting_frame = i
        while starting_frame != len(input) - 3:
            print(input[starting_frame:starting_frame+3])

            if input[starting_frame:starting_frame+3] in stop_codons:
                reading_frames.append((i, starting_frame + 3))
                break
            starting_frame += 3 # Move frame by a codon (3 bp)
    print(f"Reading frames: {reading_frames} with lengths: {[y-x for (x,y) in reading_frames]}")
        


find_reading_frames("ATGCGCTAGCGGATGATGAGGCCCTAGATGGCC")

Starting points found at [0, 12, 15, 27]
ATG
CGC
TAG
ATG
ATG
AGG
CCC
TAG
ATG
AGG
CCC
TAG
ATG
Reading frames: [(0, 9), (12, 27), (15, 27)] with lengths: [9, 15, 12]


**b) Es wurden drei reading frames bestimmt und deren $p_i$ ermittelt. Berechnen Sie nun
$P_1$ und erläutern Sie das Ergebnis.**
$$
p_1 = 0.005\\
p_2 = 0.009\\
p_3 = 0.001
$$

In [58]:
def P_i(pi, p1, p2, p3):
    return pi/(p1+p2+p3)

p1 = 0.005
p2 = 0.009
p3 = 0.001

pi_1 = P_i(p1, p1, p2, p3)
pi_2 = P_i(p2, p1, p2, p3)
pi_3 = P_i(p3, p1, p2, p3)

print(f"P_1: {pi_1}, P_2: {pi_2}, P_3: {pi_3}")


P_1: 0.33333333333333337, P_2: 0.6, P_3: 0.06666666666666667


Es gilt $P_1 \approx 33.3 \%$. Das bedeutet, dass Reading Frame 1 mit 33% das codierende Leseraster ist. Man muss es ins Verhältnis zu den Alternativhypothesen setzen, nämlich $P_2 = 60\%$ und $P_3 \approx 6.6\%$

Also ist $P_1$ tatsächlich mit geringerer Wahrscheinlichkeit das richtige Leseraster als $P_2$.

### Aufgabe 3: Genvorhersage
Suchen Sie Gene im Genom von S. cervisiae mittels einer Markovkette 3. Ordnung.
a) Laden Sie die Fasta-Dateien mit 1000 proteinkodierenden Gensequenzen (y_genes.txt)
und mit nicht-kodierenden DNA-Sequenzen (y_ncregions.txt) herunter. Schreiben Sie
eine Funktion, die aus den Sequenzen in y_genes.txt eine Markovkette 3. Ordnung für
die Gene (das Gen-Modell G) schätzt.

Die Transitionswahrscheinlichkeiten $a^{G}_{rst,u}$ können Sie wie folgt berechnen:

$$\frac{c^{G}_{rst,u}}{\sum_{l} c^{G}_{rst,l}}$$

,
wobei $c^{G}_{rst,u}$ die Zahl der $rstu$-Tetranukleotide in den Sequenzen aus
y_genes.txt ist. Geben Sie die Transitionsmatrix für das G-Modell an, runden Sie
dabei die Wahrscheinlichkeiten auf 3 Dezimalstellen.

In [ ]:
# Create our data structure
dict = {}

with open("y_genes.txt", "r") as file:
    lines = file.read().split("\n")
    first = ""
    second = ""
    for line in lines:
        if first != "" and second != "":
            dict[first] = second
            first = ""
            second = ""
        if line.startswith(">"):
            first = line
        else:
            second = line


In [ ]:
import numpy
# Transition matrix

bases = ["a", "c", "g", "t"]

transitionLabels = ""